# Import necessary libraries


In [1]:
import numpy as np
import pandas as pd
import torch
import gym
import time
import datetime
import threading
import alpaca_trade_api as tradeapi
import exchange_calendars as tc
import yfinance as yf
import matplotlib.pyplot as plt
import matplotlib.ticker as ticker
from finrl.config_tickers import DOW_30_TICKER
from finrl.config import INDICATORS
from finrl.meta.data_processor import DataProcessor
from finrl.meta.env_stock_trading.env_stocktrading_np import StockTradingEnv
from finrl.plot import backtest_stats, backtest_plot, get_daily_return, get_baseline

# Function to read the keys from keys.txt
def load_keys(file_path):
    with open(file_path, 'r') as file:
        lines = file.readlines()
        keys = {}
        for line in lines:
            if ':' in line:
                key, value = line.strip().split(':', 1)
                keys[key.strip()] = value.strip()
        return keys

# Load the keys from keys.txt
keys = load_keys('../keys.txt')

# Extract the API key and API secret
API_KEY = keys.get('keypub')
API_SECRET = keys.get('key_sec')

API_BASE_URL = 'https://paper-api.alpaca.markets'


/home/duincan/miniconda3/envs/invest/lib/python3.10/site-packages/pyfolio/pos.py:26: UserWarning: Module "zipline.assets" not found; mutltipliers will not be applied to position notionals.
  warnings.warn(


In [2]:
# Define a random agent


In [3]:
class RandomAgent:
    def __init__(self, action_dim):
        self.action_dim = action_dim

    def get_action(self, state):
        return np.random.uniform(low=-1, high=1, size=self.action_dim)


In [4]:
# Calculate state dimension and other necessary variables


In [5]:
ticker_list = DOW_30_TICKER
action_dim = len(ticker_list)
# Calculate the state dimension based on the environment specifications
state_dim = 1 + 2 + 3 * action_dim + len(INDICATORS) * action_dim

# Print the state dimension
print("State Dimension:", state_dim)


State Dimension: 333


In [6]:
# Modify the AlpacaPaperTrading class to use the random agent


In [7]:
class AlpacaPaperTrading():
    def __init__(self, ticker_list, time_interval, agent, cwd, net_dim, state_dim, action_dim, API_KEY, API_SECRET, API_BASE_URL, tech_indicator_list, turbulence_thresh=30, max_stock=1e2, latency=None):
        # Load agent
        self.agent = agent

        # Connect to Alpaca trading API
        try:
            self.alpaca = tradeapi.REST(API_KEY, API_SECRET, API_BASE_URL, 'v2')
        except:
            raise ValueError('Fail to connect Alpaca. Please check account info and internet connection.')

        # Read trading time interval
        self.time_interval = self._parse_time_interval(time_interval)

        # Read trading settings
        self.tech_indicator_list = tech_indicator_list
        self.turbulence_thresh = turbulence_thresh
        self.max_stock = max_stock

        # Initialize account
        self.stocks = np.asarray([0] * len(ticker_list))  # stocks holding
        self.stocks_cd = np.zeros_like(self.stocks)
        self.cash = None  # cash record
        self.stocks_df = pd.DataFrame(self.stocks, columns=['stocks'], index=ticker_list)
        self.asset_list = []
        self.price = np.asarray([0] * len(ticker_list))
        self.stockUniverse = ticker_list
        self.turbulence_bool = 0
        self.equities = []

    def _parse_time_interval(self, time_interval):
        if time_interval == '1s':
            return 1
        elif time_interval == '5s':
            return 5
        elif time_interval == '1Min':
            return 60
        elif time_interval == '5Min':
            return 60 * 5
        elif time_interval == '15Min':
            return 60 * 15
        else:
            raise ValueError('Time interval input is NOT supported yet.')

    def test_latency(self, test_times=10):
        total_time = 0
        for i in range(0, test_times):
            time0 = time.time()
            self.get_state()
            time1 = time.time()
            temp_time = time1 - time0
            total_time += temp_time
        latency = total_time / test_times
        print('latency for data processing: ', latency)
        return latency

    def run(self):
        orders = self.alpaca.list_orders(status="open")
        for order in orders:
            self.alpaca.cancel_order(order.id)

        # Wait for market to open.
        print("Waiting for market to open...")
        self.awaitMarketOpen()
        print("Market opened.")

        while True:
            clock = self.alpaca.get_clock()
            closingTime = clock.next_close.replace(tzinfo=datetime.timezone.utc).timestamp()
            currTime = clock.timestamp.replace(tzinfo=datetime.timezone.utc).timestamp()
            self.timeToClose = closingTime - currTime

            if self.timeToClose < (60):
                print("Market closing soon. Stop trading.")
                break

            else:
                self.trade()
                last_equity = float(self.alpaca.get_account().last_equity)
                cur_time = time.time()
                self.equities.append([cur_time, last_equity])
                time.sleep(self.time_interval)

    def awaitMarketOpen(self):
        isOpen = self.alpaca.get_clock().is_open
        while not isOpen:
            clock = self.alpaca.get_clock()
            openingTime = clock.next_open.replace(tzinfo=datetime.timezone.utc).timestamp()
            currTime = clock.timestamp.replace(tzinfo=datetime.timezone.utc).timestamp()
            timeToOpen = int((openingTime - currTime) / 60)
            print(str(timeToOpen) + " minutes til market open.")
            time.sleep(60)
            isOpen = self.alpaca.get_clock().is_open

    def trade(self):
        state = self.get_state()
        action = self.agent.get_action(state)
        action = (action * self.max_stock).astype(int)

        self.stocks_cd += 1
        if self.turbulence_bool == 0:
            min_action = 10  # stock_cd
            threads = []
            for index in np.where(action < -min_action)[0]:  # sell_index:
                sell_num_shares = min(self.stocks[index], -action[index])
                qty = abs(int(sell_num_shares))
                respSO = []
                tSubmitOrder = threading.Thread(target=self.submitOrder(qty, self.stockUniverse[index], 'sell', respSO))
                tSubmitOrder.start()
                threads.append(tSubmitOrder)
                self.cash = float(self.alpaca.get_account().cash)
                self.stocks_cd[index] = 0

            for x in threads:  # wait for all threads to complete
                x.join()

            threads = []
            for index in np.where(action > min_action)[0]:  # buy_index:
                if self.cash < 0:
                    tmp_cash = 0
                else:
                    tmp_cash = self.cash
                buy_num_shares = min(tmp_cash // self.price[index], abs(int(action[index])))
                if np.isnan(buy_num_shares):  # if buy_num_change = nan
                    qty = 0  # set to 0 quantity
                else:
                    qty = abs(int(buy_num_shares))
                respSO = []
                tSubmitOrder = threading.Thread(target=self.submitOrder(qty, self.stockUniverse[index], 'buy', respSO))
                tSubmitOrder.start()
                threads.append(tSubmitOrder)
                self.cash = float(self.alpaca.get_account().cash)
                self.stocks_cd[index] = 0

            for x in threads:  # wait for all threads to complete
                x.join()

        else:  # sell all when turbulence
            threads = []
            positions = self.alpaca.list_positions()
            for position in positions:
                if position.side == 'long':
                    orderSide = 'sell'
                else:
                    orderSide = 'buy'
                qty = abs(int(float(position.qty)))
                respSO = []
                tSubmitOrder = threading.Thread(target=self.submitOrder(qty, position.symbol, orderSide, respSO))
                tSubmitOrder.start()
                threads.append(tSubmitOrder)

            for x in threads:  # wait for all threads to complete
                x.join()

            self.stocks_cd[:] = 0

    def get_state(self):
        alpaca = AlpacaProcessor(api=self.alpaca)
        price, tech, turbulence = alpaca.fetch_latest_data(ticker_list=self.stockUniverse, time_interval='1Min', tech_indicator_list=self.tech_indicator_list)
        turbulence_bool = 1 if turbulence >= self.turbulence_thresh else 0

        turbulence = (self.sigmoid_sign(turbulence, self.turbulence_thresh) * 2 ** -5).astype(np.float32)
        tech = tech * 2 ** -7
        positions = self.alpaca.list_positions()
        stocks = [0] * len(self.stockUniverse)
        for position in positions:
            ind = self.stockUniverse.index(position.symbol)
            stocks[ind] = abs(int(float(position.qty)))

        stocks = np.asarray(stocks, dtype=float)
        cash = float(self.alpaca.get_account().cash)
        self.cash = cash
        self.stocks = stocks
        self.turbulence_bool = turbulence_bool
        self.price = price

        amount = np.array(self.cash * (2 ** -12), dtype=np.float32)
        scale = np.array(2 ** -6, dtype=np.float32)
        state = np.hstack((amount, turbulence, self.turbulence_bool, price * scale, self.stocks * scale, self.stocks_cd, tech)).astype(np.float32)
        state[np.isnan(state)] = 0.0
        state[np.isinf(state)] = 0.0
        return state

    def submitOrder(self, qty, stock, side, resp):
        if qty > 0:
            try:
                self.alpaca.submit_order(stock, qty, side, "market", "day")
                print("Market order of | " + str(qty) + " " + stock + " " + side + " | completed.")
                resp.append(True)
            except:
                print("Order of | " + str(qty) + " " + stock + " " + side + " | did not go through.")
                resp.append(False)
        else:
            print("Quantity is 0, order of | " + str(qty) + " " + stock + " " + side + " | not completed.")
            resp.append(True)

    @staticmethod
    def sigmoid_sign(ary, thresh):
        def sigmoid(x):
            return 1 / (1 + np.exp(-x * np.e)) - 0.5

        return sigmoid(ary / thresh) * thresh


In [8]:
# Initialize the random agent


In [9]:
random_agent = RandomAgent(action_dim=len(DOW_30_TICKER))


In [10]:
# Run the paper trading with the random agent


In [11]:
paper_trading_random = AlpacaPaperTrading(
    ticker_list=DOW_30_TICKER,
    time_interval='1Min',
    agent=random_agent,
    cwd=None,
    net_dim=None,
    state_dim=state_dim,
    action_dim=action_dim,
    API_KEY=API_KEY,
    API_SECRET=API_SECRET,
    API_BASE_URL=API_BASE_URL,
    tech_indicator_list=INDICATORS,
    turbulence_thresh=30,
    max_stock=1e2
)
paper_trading_random.run()


Waiting for market to open...
1964 minutes til market open.


KeyboardInterrupt: 

In [ ]:
# Plot and compare the performance of the random agent


In [ ]:
def get_trading_days(start, end):
    nyse = tc.get_calendar('NYSE')
    df = nyse.sessions_in_range(pd.Timestamp(start), pd.Timestamp(end))
    trading_days = []
    for day in df:
        trading_days.append(str(day)[:10])
    return trading_days

def alpaca_history(key, secret, url, start, end):
    api = tradeapi.REST(key, secret, url, 'v2')
    trading_days = get_trading_days(start, end)
    df = pd.DataFrame()
    for day in trading_days:
        df = pd.concat([df, api.get_portfolio_history(date_start=day, timeframe='5Min').df.iloc[:78]], ignore_index=True)
    equities = df.equity.values
    cumu_returns = equities / equities[0]
    cumu_returns = cumu_returns[~np.isnan(cumu_returns)]
    return df, cumu_returns

def DIA_history(start):
    data_df = yf.download(['^DJI'], start=start, interval="5m")
    data_df = data_df.iloc[:]
    baseline_returns = data_df['Adj Close'].values / data_df['Adj Close'].values[0]
    return data_df, baseline_returns

df_random, cumu_random = alpaca_history(key=API_KEY, secret=API_SECRET, url=API_BASE_URL, start='2022-09-01', end='2022-09-12')
df_djia, cumu_djia = DIA_history(start='2022-09-01')

returns_random = cumu_random - 1
returns_dia = cumu_djia - 1
returns_dia = returns_dia[:returns_random.shape[0]]

plt.figure(dpi=1000)
plt.grid()
plt.grid(which='minor', axis='y')
plt.title('Stock Trading (Paper trading)', fontsize=20)
plt.plot(returns_random, label='Random Agent', color='red')
plt.plot(returns_dia, label='DJIA', color='grey')
plt.ylabel('Return', fontsize=16)
plt.xlabel('Year 2021', fontsize=16)
plt.xticks(size=14)
plt.yticks(size=14)
ax = plt.gca()
ax.xaxis.set_major_locator(ticker.MultipleLocator(78))
ax.xaxis.set_minor_locator(ticker.MultipleLocator(6))
ax.yaxis.set_minor_locator(ticker.MultipleLocator(0.005))
ax.yaxis.set_major_formatter(ticker.PercentFormatter(xmax=1, decimals=2))
ax.xaxis.set_major_formatter(ticker.FixedFormatter(['', '10-19', '', '10-20', '', '10-21', '', '10-22']))
plt.legend(fontsize=10.5)
plt.savefig('papertrading_stock_random.png')
